# Task 1.2: Handling PlayerID Updates Across Related Tables

## Executive Summary

This notebook addresses the critical data architecture challenge of maintaining referential integrity when PlayerIDs are merged or updated across a multi-table database schema. We present both the theoretical framework and our practical implementation, demonstrating how cascading updates ensure data consistency in a production environment.

**Objective**: Outline a robust, repeatable approach to handle PlayerID changes that maintains accuracy across all related tables (Players, Reports, Contracts, Appearances, etc.) while ensuring the process is consistent and repeatable for future data updates.

**Requirements from PDF**:
- How to identify which tables or datasets are affected by PlayerID changes
- How to ensure that relationships between tables remain correct after updates or merges
- Steps to make this process consistent and repeatable for future data updates


In [ ]:
# Import required libraries
import pandas as pd
import json
from pathlib import Path
import sys

# Add src to path
sys.path.insert(0, str(Path('..') / 'src'))

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)


## Part 1: Theoretical Foundation - Cascading Updates in Relational Databases

### 1.1 Understanding Cascading Updates

**Cascading Updates** are a fundamental database constraint mechanism that automatically propagates changes from a primary key (or unique identifier) in a parent table to all corresponding foreign keys in child tables. This ensures referential integrity is maintained without manual intervention.

**Key Concepts**:

1. **Primary Key (Parent Table)**: The `Players` table contains the master record with `PlayerID` as the primary key
2. **Foreign Key (Child Tables)**: Related tables (`Reports`, `Contracts`, `Appearances`) reference `PlayerID` as a foreign key
3. **Cascade Behavior**: When a `PlayerID` is updated or merged in the parent table, all child table references are automatically updated

**Database-Level Implementation** (SQL):
```sql
-- Example: Foreign key constraint with CASCADE UPDATE
ALTER TABLE Reports
ADD CONSTRAINT fk_player
FOREIGN KEY (PlayerID) 
REFERENCES Players(PlayerID)
ON UPDATE CASCADE;
```

**Why This Matters**:
- **Data Integrity**: Prevents orphaned records (reports without valid player references)
- **Consistency**: Ensures all tables always reference the correct, canonical PlayerID
- **Automation**: Eliminates manual, error-prone update processes
- **Atomicity**: Changes happen transactionally - all or nothing


### 1.2 Our Implementation: Translation Lookup Table Pattern

In our ETL pipeline, we implement cascading updates using a **Translation Lookup Table** pattern, which is the industry-standard approach for data migration and ID reconciliation in data warehouses and analytics platforms.

**Architecture**:

```
┌─────────────────┐
│  Players (Raw)  │
│  - PLY_ABC123   │  ──┐
│  - PLY_XYZ789   │    │  Duplicate Detection
│  - PLY_ABC123   │  ──┘  & Canonical ID Selection
└─────────────────┘
         │
         ▼
┌─────────────────────────┐
│  player_id_map.json     │  ← Translation Lookup Table
│  {                      │
│    "PLY_XYZ789":        │
│      "PLY_ABC123"       │  ← Old ID → Canonical ID
│  }                      │
└─────────────────────────┘
         │
         ├──────────────────┐
         ▼                  ▼
┌─────────────────┐  ┌──────────────────┐
│ Players (Clean) │  │ Reports (Clean)  │
│ - PLY_ABC123    │  │ - PLY_ABC123     │  ← All IDs updated
└─────────────────┘  └──────────────────┘
```

**Key Advantages**:
1. **Auditability**: Complete change log of all ID mappings
2. **Reversibility**: Can trace back from canonical ID to original IDs
3. **Incremental Processing**: Can apply mappings to new data without reprocessing entire datasets
4. **Version Control**: JSON format allows tracking changes over time
5. **Cross-Platform**: Works with CSV, databases, APIs, and data warehouses


In [ ]:
# Load our actual ID mapping to demonstrate the pattern
BASE_DIR = Path('..')
mapping_path = BASE_DIR / 'data' / 'processed' / 'player_id_map.json'

with open(mapping_path, 'r') as f:
    id_mapping = json.load(f)

print("=" * 80)
print("TRANSLATION LOOKUP TABLE (player_id_map.json)")
print("=" * 80)
print(f"\nTotal ID Mappings: {len(id_mapping)}")
print("\nSample Mappings (showing different scenarios):")
print("-" * 80)

# Show examples
sample_mappings = list(id_mapping.items())[:10]
for old_id, new_id in sample_mappings:
    print(f"  {old_id:25s} → {new_id}")

print("\n" + "=" * 80)
print("MAPPING STATISTICS")
print("=" * 80)

# Analyze mapping patterns
mapping_types = {
    'Same ID with suffix': sum(1 for old, new in id_mapping.items() if new.startswith(old + '_')),
    'Different ID': sum(1 for old, new in id_mapping.items() if not new.startswith(old + '_')),
    'Total': len(id_mapping)
}

for mapping_type, count in mapping_types.items():
    print(f"  {mapping_type}: {count}")


## Part 2: Answering the Three Core Requirements

### 2.1 Requirement 1: How to Identify Which Tables Are Affected

**Theoretical Approach**:

In a relational database, we identify affected tables through **Foreign Key Dependencies**:

1. **Direct Dependencies**: Tables with explicit foreign key constraints referencing `Players.PlayerID`
2. **Indirect Dependencies**: Tables that reference other tables which in turn reference `Players.PlayerID`
3. **Application-Level Dependencies**: Code/APIs that use PlayerID for lookups or joins

**SQL-Based Discovery**:
```sql
-- Find all tables with foreign keys to Players
SELECT 
    TABLE_NAME,
    COLUMN_NAME,
    CONSTRAINT_NAME
FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE
WHERE REFERENCED_TABLE_NAME = 'Players'
AND REFERENCED_COLUMN_NAME = 'PlayerID';
```

**Our Practical Implementation**:

In our current dataset, we have identified the following affected tables:


In [ ]:
# Identify affected tables by examining our data structure
BASE_DIR = Path('..')

# Check what tables/files we have
data_raw = BASE_DIR / 'data' / 'raw'
data_processed = BASE_DIR / 'data' / 'processed'

print("=" * 80)
print("AFFECTED TABLES/DATASETS IDENTIFICATION")
print("=" * 80)

# Load Players to see structure
players_path = data_processed / 'players_cleaned.csv'
reporting_path = data_processed / 'reporting_cleaned.csv'

df_players = pd.read_csv(players_path)
df_reporting = pd.read_csv(reporting_path)

print("\n1. PRIMARY TABLE (Parent):")
print(f"   - Players.csv")
print(f"     • Primary Key: PlayerID")
print(f"     • Records: {len(df_players)}")
print(f"     • Unique PlayerIDs: {df_players['PlayerID'].nunique()}")

print("\n2. CHILD TABLES (Foreign Key Dependencies):")
print(f"   - ReportingInsight.csv")
print(f"     • Foreign Key: PlayerID (references Players.PlayerID)")
print(f"     • Records: {len(df_reporting)}")
print(f"     • Unique PlayerIDs referenced: {df_reporting['PlayerID'].nunique()}")

# Check for orphaned references
players_ids = set(df_players['PlayerID'].unique())
reporting_ids = set(df_reporting['PlayerID'].dropna().unique())
orphaned = reporting_ids - players_ids

print(f"\n3. REFERENTIAL INTEGRITY CHECK:")
print(f"   • PlayerIDs in Reporting: {len(reporting_ids)}")
print(f"   • PlayerIDs in Players: {len(players_ids)}")
print(f"   • Orphaned PlayerIDs: {len(orphaned)}")
if len(orphaned) == 0:
    print(f"   ✅ Status: PASS - All references valid")
else:
    print(f"   ⚠️  Status: WARNING - {len(orphaned)} orphaned references found")

print("\n4. POTENTIAL FUTURE TABLES (Not in current dataset but expected in production):")
print("   - Contracts (ContractID, PlayerID, StartDate, EndDate, Salary)")
print("   - Appearances (AppearanceID, PlayerID, MatchID, Minutes, Goals)")
print("   - Injuries (InjuryID, PlayerID, InjuryType, RecoveryDate)")
print("   - Transfers (TransferID, PlayerID, FromClub, ToClub, TransferFee)")
print("   - Medical Records (MedicalID, PlayerID, ExaminationDate, Status)")


### 2.2 Requirement 2: Ensuring Relationships Remain Correct

**Theoretical Framework**:

Maintaining referential integrity requires three critical mechanisms:

#### A. Foreign Key Constraints

**Database-Level** (Production SQL Database):
```sql
-- Enforce referential integrity at database level
ALTER TABLE Reports
ADD CONSTRAINT fk_reports_player
FOREIGN KEY (PlayerID) 
REFERENCES Players(PlayerID)
ON UPDATE CASCADE
ON DELETE RESTRICT;
```

**ETL-Level** (Our Current Implementation):
- Validation step after ID mapping application
- Orphaned record detection and resolution
- Transactional processing (all-or-nothing updates)

#### B. Transaction Atomicity

All ID updates must occur within a single transaction to ensure:
- **Consistency**: Either all tables are updated, or none are
- **Isolation**: No partial updates visible to other processes
- **Durability**: Changes are permanent once committed

#### C. Validation and Verification

Post-update validation ensures:
1. All foreign key references point to valid PlayerIDs
2. No orphaned records exist
3. Mapping completeness (all old IDs have corresponding new IDs)

**Our Implementation**:


In [ ]:
# Demonstrate our referential integrity validation
from clean_reporting import validate_referential_integrity

# Load cleaned data
df_players_clean = pd.read_csv(players_path)
df_reporting_clean = pd.read_csv(reporting_path)

# Perform validation
integrity_check = validate_referential_integrity(df_reporting_clean, df_players_clean)

print("=" * 80)
print("REFERENTIAL INTEGRITY VALIDATION")
print("=" * 80)
print(f"\nStatus: {integrity_check['integrity_status']}")
print(f"Unique PlayerIDs in Reporting: {integrity_check['total_unique_player_ids_in_reporting']}")
print(f"Unique PlayerIDs in Players: {integrity_check['total_unique_player_ids_in_players']}")
print(f"Orphaned PlayerIDs: {integrity_check['orphaned_count']}")
print(f"Orphaned Rows: {integrity_check['orphaned_rows']}")

if integrity_check['integrity_status'] == 'PASS':
    print("\n✅ SUCCESS: All relationships maintained correctly!")
    print("   Every PlayerID in ReportingInsight exists in Players table.")
else:
    print(f"\n⚠️  WARNING: {integrity_check['orphaned_count']} orphaned references detected")
    if integrity_check['orphaned_player_ids']:
        print("   Orphaned IDs:", integrity_check['orphaned_player_ids'][:10])

# Show how ID mapping was applied
print("\n" + "=" * 80)
print("ID MAPPING APPLICATION PROCESS")
print("=" * 80)

# Count how many reports were affected by ID mappings
mapped_ids_in_reporting = set(id_mapping.values())
reports_with_mapped_ids = df_reporting_clean[df_reporting_clean['PlayerID'].isin(mapped_ids_in_reporting)]

print(f"\nReports affected by ID mappings: {len(reports_with_mapped_ids)}")
print(f"Percentage of total reports: {len(reports_with_mapped_ids) / len(df_reporting_clean) * 100:.1f}%")

# Show example
if len(reports_with_mapped_ids) > 0:
    print("\nExample: Reports with mapped PlayerIDs:")
    example = reports_with_mapped_ids[['PlayerID', 'PlayerName', 'PerformanceGrade', 'PotentialGrade']].head(5)
    print(example.to_string(index=False))


### 2.3 Requirement 3: Making the Process Repeatable and Consistent

**Architecture Principles for Repeatability**:

#### A. ETL Pipeline Automation

Our pipeline follows the **Extract-Transform-Load (ETL)** pattern with these characteristics:

1. **Idempotency**: Running the pipeline multiple times produces the same result
2. **Incremental Processing**: Can handle new data without reprocessing entire datasets
3. **Version Control**: ID mappings are versioned and auditable
4. **Error Handling**: Graceful failure with rollback capabilities

#### B. Logging and Audit Trail

Every ID change is logged with:
- **Timestamp**: When the change occurred
- **Source**: Which process/script made the change
- **Reason**: Why the ID was changed (duplicate detection, data quality, etc.)
- **Impact**: How many records were affected

#### C. Configuration Management

The process is driven by configuration, not hardcoded logic:
- Mapping rules are data-driven (stored in JSON)
- Cleaning rules are parameterized
- Validation thresholds are configurable

**Our Implementation Structure**:


In [ ]:
# Demonstrate the repeatable pipeline structure
print("=" * 80)
print("REPEATABLE PIPELINE ARCHITECTURE")
print("=" * 80)

print("\n1. PIPELINE ORCHESTRATION (src/pipeline.py):")
print("   ├── Step 1: Clean Players Data")
print("   │   └── Generates: players_cleaned.csv + player_id_map.json")
print("   │")
print("   ├── Step 2: Clean Reporting Data")
print("   │   ├── Loads: player_id_map.json")
print("   │   ├── Applies: ID mappings to ReportingInsight")
print("   │   └── Generates: reporting_cleaned.csv")
print("   │")
print("   └── Step 3: Generate Audit Log")
print("       └── Generates: cleaning_log.md (before/after metrics)")

print("\n2. ID MAPPING APPLICATION (src/clean_reporting.py):")
print("   - Function: apply_id_mapping()")
print("   - Input: Reporting DataFrame + ID Mapping Dictionary")
print("   - Process: Lookup and replace old IDs with canonical IDs")
print("   - Output: Updated Reporting DataFrame with mapped IDs")

# Show the actual mapping application code pattern
print("\n3. CODE PATTERN (Pseudocode):")
print("""
def apply_id_mapping(df_reporting, id_mapping):
    # Create lookup function
    def map_player_id(player_id):
        if player_id in id_mapping:
            return id_mapping[player_id]  # Return canonical ID
        return player_id  # Keep original if not in mapping
    
    # Apply mapping to all rows
    df_reporting['PlayerID'] = df_reporting['PlayerID'].apply(map_player_id)
    
    # Return updated dataframe
    return df_reporting
""")

print("\n4. VALIDATION CHECKPOINTS:")
print("   ✅ Pre-processing: Validate input data structure")
print("   ✅ Post-mapping: Verify all IDs exist in Players table")
print("   ✅ Final: Generate integrity report")


## Part 3: Production-Ready Implementation Strategy

### 3.1 Database Migration Strategy (For Production SQL Database)

When moving from CSV-based ETL to a production database, the implementation would follow this pattern:

**Phase 1: Preparation**
```sql
-- Create staging table for ID mappings
CREATE TABLE player_id_mappings (
    old_player_id VARCHAR(50) PRIMARY KEY,
    new_player_id VARCHAR(50) NOT NULL,
    mapping_reason VARCHAR(255),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    INDEX idx_new_id (new_player_id)
);
```

**Phase 2: Apply Mappings**
```sql
-- Update Reports table using JOIN with mapping table
UPDATE Reports r
INNER JOIN player_id_mappings m ON r.PlayerID = m.old_player_id
SET r.PlayerID = m.new_player_id;

-- Similar updates for Contracts, Appearances, etc.
UPDATE Contracts c
INNER JOIN player_id_mappings m ON c.PlayerID = m.old_player_id
SET c.PlayerID = m.new_player_id;
```

**Phase 3: Validation**
```sql
-- Verify no orphaned references
SELECT COUNT(*) as orphaned_count
FROM Reports r
LEFT JOIN Players p ON r.PlayerID = p.PlayerID
WHERE p.PlayerID IS NULL;
-- Should return 0
```

### 3.2 Incremental Update Handling

For ongoing operations, new data can be processed incrementally:

1. **New Player Data**: Run duplicate detection only on new records + existing cleaned data
2. **New Reports**: Apply existing ID mappings to new reports before insertion
3. **ID Mapping Updates**: Append new mappings to existing mapping table (never overwrite)

### 3.3 Rollback Strategy

Maintain ability to reverse changes:
- Keep original PlayerIDs in audit columns
- Store mapping history with timestamps
- Enable point-in-time recovery

**Example Audit Schema**:
```sql
ALTER TABLE Reports ADD COLUMN original_player_id VARCHAR(50);
ALTER TABLE Reports ADD COLUMN player_id_updated_at TIMESTAMP;
```


## Part 4: Best Practices and Lessons Learned

### 4.1 Key Design Decisions

1. **Canonical ID Selection**: We prioritize data completeness and CurrentTeam presence when selecting which PlayerID to keep. This ensures we retain the most valuable record.

2. **Orphaned Record Handling**: Rather than deleting orphaned records, we create minimal player records from Reporting data. This preserves data lineage and auditability.

3. **Mapping Persistence**: JSON format allows easy version control, human readability, and cross-platform compatibility.

### 4.2 Performance Considerations

- **Lookup Efficiency**: Dictionary/hash map provides O(1) lookup time for ID mappings
- **Batch Processing**: Apply mappings to entire DataFrames rather than row-by-row
- **Caching**: ID mapping loaded once and reused across all child tables

### 4.3 Risk Mitigation

- **Validation Gates**: Multiple checkpoints prevent bad data from propagating
- **Audit Logging**: Complete trail of all changes for compliance and debugging
- **Testing**: Unit tests for mapping logic, integration tests for full pipeline

## Conclusion

Our implementation demonstrates a production-ready approach to handling PlayerID updates that:

✅ **Identifies Affected Tables**: Through foreign key analysis and data profiling  
✅ **Maintains Relationships**: Via validation, transaction atomicity, and integrity checks  
✅ **Ensures Repeatability**: Through automated ETL pipelines, versioned mappings, and comprehensive logging  

This architecture scales from CSV-based analytics to enterprise database systems while maintaining data quality and auditability.
